In [2]:
import pytorch_lightning as pl
import torch
import Operator as op
import utils
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
lattice_sites = 3
hamiltonian = []
h = -1
'''
for l in range(lattice_sites):
  hamiltonian = Sx(l)* (h) + Sz(l) * Sz((l+1) % lattice_sites) + hamiltonian
print_op_list(hamiltonian)
'''
hamiltonian = op.Sx(2) + ([op.Sz(1)*op.Sy(2)] + hamiltonian)
op.print_op_list(hamiltonian)

adding to sequence
Hamiltonan = Sx_2 + Sz_1 * Sy_2


In [17]:
import torch
from torch.nn import functional as F
import pytorch_lightning as pl
from torch import nn

class Model(pl.LightningModule):

  def __init__(self, lattice_sites):
    super().__init__()
    self.lattice_net = nn.Sequential(
      nn.Conv1d(1, 8, kernel_size=2, padding=1, padding_mode='circular'),
      nn.ReLU(),
      nn.Conv1d(8, 16, kernel_size=2, padding=1, padding_mode='circular'),
      nn.Flatten()
    )
    
    self.tNN = nn.Sequential(
      nn.Linear(2, 16),
      nn.ReLU(),
      nn.Linear(16, 32),
      nn.ReLU(),
      nn.Linear(32, 64)
    )

    self.psi = nn.Sequential(
      nn.Linear( 64 + 16 * ( lattice_sites + 2 ), 256 ),
      nn.ReLU(),
      nn.Linear(256, 128),
      nn.ReLU(),
      nn.Linear(128,1)
    )


  def forward(self, spins, t, h):
    input = torch.unsqueeze(spins, 1)
    lat_out = self.lattice_net(input)
    tNN_inp = torch.cat((t,h), dim = 1)
    t_out = self.tNN(tNN_inp)
    wave_fun = self.psi(torch.cat((lat_out, t_out), dim=1))
    return wave_fun

model = Model(lattice_sites)
inp = torch.ones((2, lattice_sites))
t_arr = torch.ones((2,1))
h_arr = torch.ones((2,1))
print(model(inp, t_arr, h_arr))

tensor([[-0.0113],
        [-0.0113]], grad_fn=<AddmmBackward>)


In [4]:
from timeit import default_timer as timer

map = utils.get_map(hamiltonian, lattice_sites)
map = map.to(device)
print("map: ", map)

mat_els = utils.get_total_mat_els(hamiltonian, lattice_sites)
mat_els = mat_els.to(device)
print("mat els: ", mat_els)

#1.dim: Batch
#2.dim: lattice sites
#s_config = get_all_spin_configs(lattice_sites)
s_config = torch.tensor([1,1,1]).reshape(1,3)
s_config = (s_config.type(torch.float32)).to(device)
print("spin config: ", s_config)

start = timer()
s_p = utils.get_sp(s_config, map)
psi_s = model(s_config)
psi_sp = model(s_p.reshape(-1, lattice_sites)).reshape(s_p.shape[0], s_p.shape[1])
print(psi_sp.shape, psi_s.shape, s_config.shape)
O_loc = utils.calc_Oloc(psi_sp, mat_els, s_config)
end = timer()

print(f"time to calculate O_loc: {end - start:.2e}") 

map:  tensor([[[ 1,  1, -1],
         [ 1,  1, -1]]], device='cuda:0', dtype=torch.int8)
mat els:  tensor([[[[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j,  1.+0.j,  0.+1.j]],

         [[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j, -1.+0.j, -0.-1.j]]]], device='cuda:0')
spin config:  tensor([[1., 1., 1.]], device='cuda:0')
torch.Size([1, 2]) torch.Size([1, 1]) torch.Size([1, 3])
time to calculate O_loc: 4.15e-02
